## Graph Algorithm

#### Create Adjacency List

In [ ]:
# G = (V, E)
from collections import namedtuple

Graph = namedtuple("Graph", ["nodes", "edges","isDirected"])
nodes = ['s','a','x','z','d','c','f','v']
edges = [
    ('s','a'),
    ('a','z'),
    ('s','x'),
    ('x','c'),
    ('d','c'),
    ('d','f'),
    ('c','f'),
    ('c','v'),
]

G = Graph(nodes,edges,isDirected = False)

def adjacency_dict(graph):
    """
    Returns the adjacency list representation
    of the graph.
    """
    adj = {node: [] for node in graph.nodes}
    for edge in graph.edges:
        node1, node2 = edge[0], edge[1]
        adj[node1].append(node2)
        if graph.isDirected == False:
            adj[node2].append(node1)
    return adj

Adj = adjacency_dict(G)
Adj

{'s': ['a', 'x'],
 'a': ['s', 'z'],
 'x': ['s', 'c'],
 'z': ['a'],
 'd': ['c', 'f'],
 'c': ['x', 'd', 'f', 'v'],
 'f': ['d', 'c'],
 'v': ['c']}

#### Breadth-First Search

In [ ]:
def BFS(Adj,s):
    level = {s:0}
    parent = {s: None}
    i = 1
    frontier = [s]
    while frontier:
        next = []
        for u in frontier:
            for v in Adj[u]:
                if v not in level:
                    level[v] = i
                    parent[v] = u
                    next.append(v)
        frontier = next
        i += 1
    return parent

def unweighted_shortest_path(Adj,s,t):
    parent = BFS(Adj,s)
    if parent is None:
        return None
    path = [t]
    i = t
    while i != s:
        i = parent[i]
        path.append(i)
    path.reverse()
    return path

unweighted_shortest_path(Adj,'s','v')

['s', 'x', 'c', 'v']

#### Dept-First Search

In [ ]:
def DFS_visit(Adj,parent,order,s):
    for v in Adj[s]:
        if v not in parent:
            parent[v] = s
            DFS_visit(Adj,parent,order,v)
    order.append(s)

def DFS(Adj,s):
    parent = {}
    order = []
    for s in Adj:
        if s not in parent:
            parent[s] = None
            DFS_visit(Adj,parent,order,s) ### Remember: python just don't bring the reference, i.e. parent[s] = None to the next call function.
                                            #### So, we need to add that parameter explicitely.
    order.reverse()
    return parent, order

parent, order = DFS(Adj,'s')
parent, order

({'s': None,
  'a': 's',
  'z': 'a',
  'x': 's',
  'c': 'x',
  'd': 'c',
  'f': 'd',
  'v': 'c'},
 ['s', 'x', 'c', 'v', 'd', 'f', 'a', 'z'])

#### Weighted Shortest Paths in a DAG

In [ ]:
import numpy as np

w = {0:{1:5,2:3},
      1:{2:2,3:6},
      2:{3:7,4:4,5:2},
      3:{4:-1,5:1},
      4:{5:-2},
      5:{}}

Adj = {0:[1,2],
       1:[2,3],
       2:[3,4,5],
       3:[4,5],
       4:[5],
       5:[]}

V = [0,1,2,3,4,5]

def init_single_source(V,s):
    d = {}
    parent = {}
    for v in V:
        d[v] = float('inf')
        parent[v] = None
    d[s] = 0
    return parent, d

def relax(u,v,w,parent,d):
    if d[v] > d[u] + w[u][v]:
        d[v] = d[u] + w[u][v]
        parent[v] = u
    return parent,d

def DAG_shortest_paths(Adj,w,V,s):
    _, order = DFS(V,Adj,s)
    parent, d = init_single_source(V,s)
    for u in order:
        for v in Adj[u]:
            relax(u,v,w,parent,d)
    return d,parent

DAG_shortest_paths(Adj,w,V,s=1)

({0: inf, 1: 0, 2: 2, 3: 6, 4: 5, 5: 3},
 {0: None, 1: None, 2: 1, 3: 1, 4: 3, 5: 4})

#### Bellman-Ford Algorithm

In [ ]:
Adj2 = {0:[1,3],
       1:[2,3,4],
       2:[1],
       3:[2,4],
       4:[0,2]}

w2 = {0:{1:6,3:7},
     1:{2:5,3:8,4:-4},
     2:{1:-2},
     3:{2:-3,4:9},
     4:{0:2,2:7}}

V2 = [0,1,2,3,4]

def bellman_ford(V,Adj,w,s):
    parent, d = init_single_source(V,s)
    for k in range(len(V)-1): ### Why?
        for u in V:
            for v in Adj[u]:
                if d[v] > d[u] + w[u][v]:
                    d[v] = d[u] + w[u][v]
                    parent[v] = u
    for u in V:
        for v in Adj[u]:
            if d[v] > d[u] + w[u][v]:
                return False
                # raise Exception("Oops! There's negative weithed cycle.")
    return d, parent
    # return f"Weight: {d}, Parent: {parent}"

bellman_ford(V2,Adj2,w2,0)

({0: 0, 1: 2, 2: 4, 3: 7, 4: -2}, {0: None, 1: 2, 2: 3, 3: 0, 4: 1})

#### Dijkstra Algorithm

In [ ]:
heap_size = 10

def parent(i):
    return i//2
def left(i):
    return 2*i
def right(i):
    return 2*i + 1

def min_heapify(A,heap_size,i):
    l = left(i)
    r = right(i)
    if l <= heap_size and A[l] < A[i]:
        largest = l
    else:
        largest = i
    if r <= heap_size and A[r] < A[largest]:
        largest = r
    if largest != i:
        A[largest], A[i] = A[i], A[largest]
        min_heapify(A,heap_size,largest)

def build_min_heap(A):
    heap_size = len(A)
    for i in range(len(A)//2,0,-1):
        min_heapify(A,heap_size,i)

def heap_sort(A):
    heap_size = len(A)
    build_min_heap(A)
    for i in range(len(A),1,-1):
        A[1],A[i] = A[i],A[1]
        heap_size = heap_size - 1
        min_heapify(A,heap_size,1)

def heap_extract_min(A):
    global heap_size
    if heap_size < 1:
        raise KeyError("heap underflow")
    min = A[1]
    A[1] = A[heap_size]
    heap_size = heap_size - 1
    min_heapify(A,heap_size,1)
    return min

def heap_decrease(A,i,key):
    if key > A[i]:
        raise ValueError("new key is lager than the current key")
    A[i] = key
    while i > 1 and A[i] < A[parent(i)]:
        A[i], A[parent(i)] = A[parent(i)], A[i]
        i = parent(i)

def heap_insert(A,key):
    global heap_size
    heap_size = heap_size + 1
    A[heap_size] = float('inf')
    heap_decrease(A,heap_size,key)

A = {1:16,2:4,3:10,4:14,5:7,6:9,7:3,8:2,9:8,10:1}
build_min_heap(A)
heap_insert(A,5)
heap_insert(A,17)
A

{1: 1, 2: 2, 3: 3, 4: 8, 5: 4, 6: 9, 7: 10, 8: 14, 9: 16, 10: 7, 11: 5, 12: 17}

In [ ]:
class ArrayPriorityQueue:

    def __init__(self):
        self.A = {}
    
    def insert(self,label,key):
        self.A[label] = key
    
    def extract_min(self):
        min_label = None
        for label in self.A:
            if min_label is None or self.A[label] < self.A[min_label]:
                min_label = label
        del self.A[min_label]
        return min_label

    def decrease(self,label,key):
        if label in self.A and self.A[label] > key:
            self.A[label] = key

w = {0:{1:10,3:5},
      1:{2:1,3:2},
      2:{4:4},
      3:{1:3,2:9,4:2},
      4:{0:7,2:6}}

Adj = {0:[1,3],
       1:[2,3],
       2:[4],
       3:[1,2,4],
       4:[0,2]}

V = [0,1,2,3,4]

def dijkstra(V,Adj,w,s):
       parent, d = init_single_source(V,s)
       Q = ArrayPriorityQueue()
       for v in V:
              Q.insert(v,d[v])
       for _ in V:
              u = Q.extract_min()
              for v in Adj[u]:
                     if d[v] > d[u] + w[u][v]:
                            d[v] = d[u] + w[u][v]
                            parent[v] = u
                            Q.decrease(v,d[v])
       return d,parent

dijkstra(V,Adj,w,s=0)

({0: 0, 1: 8, 2: 9, 3: 5, 4: 7}, {0: None, 1: 3, 2: 1, 3: 0, 4: 3})

#### APSP (Runtime: O(V^4))

In [ ]:
inf = float('inf')
from copy import copy, deepcopy

w = [[0,3,8,inf,-4],
     [inf,0,inf,1,7],
     [inf,4,0,inf,inf],
     [2,inf,-5,0,inf],
     [inf,inf,inf,6,0]]

def slow_APSP(w):
    n = len(w[0])
    d = [rows[:] for rows in w]
    for m in range(n):
        for u in range(n):
            for v in range(n):
                for x in range(n): # Guessing: last edge in the path u ~~~> x -> v; Since we don't what is the last edge, so we try them all.
                    if d[u][v] > d[u][x] + d[x][v]: # Shortest path should follow triangle inequality
                        d[u][v] = d[u][x] + d[x][v]
    return d

slow_APSP(w) # O(V^4) Negative weighted exist <=> negative diagonal entry

[[0, 1, -3, 2, -4],
 [3, 0, -4, 1, -1],
 [7, 4, 0, 5, 3],
 [2, -1, -5, 0, -2],
 [8, 5, 1, 6, 0]]

#### Floyd-Warshall Algorithm (Application: Dense Graph; Runtime: O((V^3))

In [ ]:
inf = float('inf')
from copy import copy, deepcopy

w = [[0,3,8,inf,-4],
     [inf,0,inf,1,7],
     [inf,4,0,inf,inf],
     [2,inf,-5,0,inf],
     [inf,inf,inf,6,0]]

def floyd_warshall(w):
    n = len(w[0])
    c = [row[:] for row in w]
    # c = list(map(lambda w:w[:],w))
    # c = deepcopy(w)
    for k in range(n):
        for u in range(n):
            for v in range(n):
                if c[u][v] > c[u][k] + c[k][v]:
                    c[u][v] = c[u][k] + c[k][v]
    return c

floyd_warshall(w)

[[0, 1, -3, 2, -4],
 [3, 0, -4, 1, -1],
 [7, 4, 0, 5, 3],
 [2, -1, -5, 0, -2],
 [8, 5, 1, 6, 0]]

#### Johnson Algorithm (Application: Sparse Graph; Runtime: O((VE + V^2Log(V)))

In [ ]:
inf = float('inf')

w = {1:{2:3,3:8,5:-4},
     2:{4:1,5:7},
     3:{2:4},
     4:{1:2,3:-5},
     5:{4:6}}

Adj = {1:[2,3,5],
     2:[4,5],
     3:[2],
     4:[1,3],
     5:[4]}

V = [1,2,3,4,5]

def jonhson(Adj,w,V,s=0):
     Vp = [s] + V
     n = len(Vp)
     Adj_p = {0:V}
     Adj_p.update(Adj)
     wp = {0:{i:0 for i in V}}
     wp.update(w)
     if bellman_ford(Vp,Adj_p,wp,s=0) == False:
          raise Exception("There's negative weighted cycle.")
     else:
          h, _ =  bellman_ford(Vp,Adj_p,wp,s=0)
          w_new = {j:{i:None for i in Vp} for j in Vp}
          for u in Vp:
               for v in Adj_p[u]:
                    w_new[u][v] = wp[u][v] + h[u] - h[v]
          d = [[inf for i in Vp] for j in Vp]
          delta = {}
          for u in V:
               delta[u], _ = dijkstra(V,Adj,w_new,u)
               for v in V:
                    d[u][v] = delta[u][v] + h[v] - h[u]
          return d

jonhson(Adj,w,V,s=0)

[[inf, inf, inf, inf, inf, inf],
 [inf, 0, 1, -3, 2, -4],
 [inf, 3, 0, -4, 1, -1],
 [inf, 7, 4, 0, 5, 3],
 [inf, 2, -1, -5, 0, -2],
 [inf, 8, 5, 1, 6, 0]]

#### Kruskal's Algorithm (Find Minimum Spanning Tree; Runtime: Base on sorting(E) algorithm O(Elog(V) if sorting take O(ELogE)))

In [ ]:
### Disjoint-Set Data Structure (or called Union-Find Data Structure or Merge-Find Set)

class DisjointSetForest():
    rank = {}
    p = {}

    def make_set(self,x):
        self.p[x] = x
        self.rank[x] = 0

    def union(self,u,v):
        x,y = self.find_set(u),self.find_set(v)
        if self.rank[x] > self.rank[y]:
            self.p[y] = x
        else:
            self.p[x] = y
            if self.rank[x] == self.rank[y]:
                self.rank[y] = self.rank[y] + 1

    def find_set(self,x):
        if x != self.p[x]:
            self.p[x] = self.find_set(self.p[x])
        return self.p[x]

### Radix Sorting: Sorting integer in linear time O(E) or O(ELog(E)) otherwise


### Kruskal's Algorithm
inf = float("inf")

V = [i for i in range(9)]

w = {0:{1:4,6:8},
    1:{0:4,2:8,6:11},
    2:{1:8,3:7,4:2,8:4},
    3:{2:7,5:9,8:14},
    4:{2:2,6:7,7:6},
    5:{3:9,8:10},
    6:{0:8,1:11,4:7,7:1},
    7:{4:6,6:1,8:2},
    8:{2:4,3:14,5:10,7:2}}

Adj = {0:[1,6],
    1:[0,2,6],
    2:[1,3,4,8],
    3:[2,5,8],
    4:[2,6,7],
    5:[3,8],
    6:[0,1,4,7],
    7:[4,6,8],
    8:[2,3,5,7]}

E = [(0,1,4),(0,6,8),
    (1,0,4),(1,2,8),(1,6,11),
    (2,1,8),(2,3,7),(2,4,2),(2,8,4),
    (3,2,7),(3,5,9),(3,8,14),
    (4,2,2),(4,6,7),(4,7,6),
    (5,3,9),(5,8,10),
    (6,0,8),(6,1,11),(6,4,7),(6,7,1),
    (7,4,6),(7,6,1),(7,8,2),
    (8,2,4),(8,3,14),(8,5,10),(8,7,2)]

def kruskal(V,E):
    MST = []
    S = DisjointSetForest()
    for v in V:
        S.make_set(v)
    E.sort(key=lambda x: x[2])
    for u,v,w in E:
        if S.find_set(u) != S.find_set(v):
            MST.append((u,v,w))
            S.union(u,v)
    return MST

kruskal(V,E)

[(6, 7, 1),
 (2, 4, 2),
 (7, 8, 2),
 (0, 1, 4),
 (2, 8, 4),
 (2, 3, 7),
 (0, 6, 8),
 (3, 5, 9)]

#### Prim's Algorithm (Runtime: Same as Dijkstra's Algorithm)

In [ ]:
inf = float("inf")

w = {0:{1:4,6:8},
    1:{0:4,2:8,6:11},
    2:{1:8,3:7,4:2,8:4},
    3:{2:7,5:9,8:14},
    4:{2:2,6:7,7:6},
    5:{3:9,8:10},
    6:{0:8,1:11,4:7,7:1},
    7:{4:6,6:1,8:2},
    8:{2:4,3:14,5:10,7:2}}

# w = {1:{2:2,3:3,4:4},
#           2:{1:2},
#           3:{1:3,7:5,4:6},
#           4:{1:4,3:6,5:7},
#           5:{4:7,6:8},
#           6:{5:8,7:9},
#           7:{6:9,3:5}}

def prim(w,r):
    S = []
    d = {}
    parent = {}
    for u in w.keys():
        d[u] = inf
        parent[u] = None
    d[r] = 0
    Q = ArrayPriorityQueue()
    for v in w.keys():
        Q.insert(v,d[v])
    for _ in w.keys():
        u = Q.extract_min()
        S.append(u) # Delete u from Q and add it into S
        for v in w[u]:
            if v not in S and d[v] > w[u][v]: # Must be guarantee that the new edge was not add before.
                d[v] = w[u][v]
                parent[v] = u
                Q.decrease(v,d[v])
    return sum(d.values()), parent

prim(w,r=1) # the root r could be any vertex of the graph

(37, {0: 1, 1: None, 2: 1, 3: 2, 4: 2, 5: 3, 6: 7, 7: 8, 8: 2})